In [ ]:
#1. Instalarea pachetelor
!pip install pandas numpy matplotlib seaborn scikit-learn xgboost catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.5 MB/s eta 0:00:00


In [ ]:
#2. Importul bibliotecilor necesare

# Importăm bibliotecile de bază
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Librăriile pentru modelare și evaluare
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [ ]:
#3. Încărcarea datelor
# Linkurile raw GitHub pentru fișierele de train și test
train_url = 'https://raw.githubusercontent.com/antoniamitu/proiect_econometrie/main/data/processed/train_data_raw.csv'
test_url  = 'https://raw.githubusercontent.com/antoniamitu/proiect_econometrie/main/data/processed/test_data_raw.csv'

In [ ]:
# Citim datele
train_df = pd.read_csv(train_url)
test_df  = pd.read_csv(test_url)

In [ ]:
# Inspectăm primele rânduri
display(train_df.head())
display(test_df.head())

,iso_code,country,Y,X1,X2,X3,X4,X5,D
0,DZA,Algeria,43.076841,4.947281,11.573331,3.77,4.10,14782.200300,LowFreedom
1,AGO,Angola,20.499484,7.538525,16.225101,3.53,5.00,7397.486427,LowFreedom
2,ARM,Armenia,40.072624,24.165340,69.638205,5.64,6.39,17886.178390,HighFreedom
3,AUS,Australia,84.349080,117.651030,19.317805,8.69,8.24,59883.647040,HighFreedom
4,AUT,Austria,28.233324,11.453004,176.327196,8.49,6.81,65661.452190,HighFreedom


,iso_code,country,Y,X1,X2,X3,X4,X5,D
0,ALB,Albania,64.543925,17.440075,37.185198,5.58,6.90,17112.937940,HighFreedom
1,ARG,Argentina,23.290508,12.997021,75.316584,5.41,5.00,27627.963480,LowFreedom
2,BGD,Bangladesh,39.523060,13.197383,9.282841,3.44,5.43,7888.163891,LowFreedom
3,BLZ,Belize,44.468487,45.277582,14.749515,3.85,6.98,12569.817940,LowFreedom
4,BWA,Botswana,23.716286,38.148957,8.309073,6.08,5.83,18647.373460,LowFreedom


In [ ]:
# Afișăm dimensiunile pentru a vedea numărul de observații
print("Dimensiuni set train:", train_df.shape)
print("Dimensiuni set test:", test_df.shape)

Dimensiuni set train: (111, 9)
Dimensiuni set test: (24, 9)


In [ ]:
#4. Preprocesarea datelor

#4.1 Verificarea valorilor lipsă
# Verificăm câte valori lipsă există în fiecare coloană
print("Valori lipsă în train:")
print(train_df.isnull().sum())
print("\nValori lipsă în test:")
print(test_df.isnull().sum())

Valori lipsă în train:
iso_code    0
country     0
Y           0
X1          0
X2          0
X3          0
X4          0
X5          0
D           0
dtype: int64

Valori lipsă în test:
iso_code    0
country     0
Y           0
X1          0
X2          0
X3          0
X4          0
X5          0
D           0
dtype: int64


In [ ]:
#4.2 Separarea variabilei țintă și a predictorilor
# Definim lista predictorilor (corelată cu structura dataset‑ului)
feature_cols = ['X1', 'X2', 'X3', 'X4', 'X5', 'D']

In [ ]:
# Separăm variabila dependentă (Y) și matricea predictorilor
X_train = train_df[feature_cols].copy()
y_train = train_df['Y'].copy()

In [ ]:
X_test = test_df[feature_cols].copy()
y_test = test_df['Y'].copy()

In [ ]:
# Convertim coloana categorică 'D' în format numeric folosind one-hot encoding
X_train = pd.get_dummies(X_train, columns=['D'], drop_first=True, dtype=int)
X_test = pd.get_dummies(X_test, columns=['D'], drop_first=True, dtype=int)

In [ ]:
#4.3 Standardizarea predictorilor
# Standardizăm predictorii pentru că modelele de regularizare sunt sensibile la scală
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [ ]:
#5. Modele de regularizare
#5.1 Definirea gamei de hiperparametri

# Definim gama de valori alpha (parametrul de penalizare) pe o scară logaritmică
alphas = np.logspace(-4, 4, 100)  # de la 1e-4 la 1e+4

In [ ]:
#5.2 Ridge Regression cu cross‑validation
# Antrenăm regresia Ridge folosind cross‑validation
ridge_cv = RidgeCV(alphas=alphas, cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X_train_scaled, y_train)

RidgeCV(alphas=array([1.00000000e-04, 1.20450354e-04, 1.45082878e-04, 1.74752840e-04,
       2.10490414e-04, 2.53536449e-04, 3.05385551e-04, 3.67837977e-04,
       4.43062146e-04, 5.33669923e-04, 6.42807312e-04, 7.74263683e-04,
       9.32603347e-04, 1.12332403e-03, 1.35304777e-03, 1.62975083e-03,
       1.96304065e-03, 2.36448941e-03, 2.84803587e-03, 3.43046929e-03,
       4.13201240e-03, 4.97702356e-0...
       1.38488637e+02, 1.66810054e+02, 2.00923300e+02, 2.42012826e+02,
       2.91505306e+02, 3.51119173e+02, 4.22924287e+02, 5.09413801e+02,
       6.13590727e+02, 7.39072203e+02, 8.90215085e+02, 1.07226722e+03,
       1.29154967e+03, 1.55567614e+03, 1.87381742e+03, 2.25701972e+03,
       2.71858824e+03, 3.27454916e+03, 3.94420606e+03, 4.75081016e+03,
       5.72236766e+03, 6.89261210e+03, 8.30217568e+03, 1.00000000e+04]),
        cv=5, scoring='neg_mean_squared_error')

In [ ]:
# Cel mai bun alpha găsit
ridge_alpha = ridge_cv.alpha_
print(f"Alpha optim pentru Ridge: {ridge_alpha:.5f}")

Alpha optim pentru Ridge: 31.25716


In [ ]:
# Predicții pe setul de test
ridge_pred = ridge_cv.predict(X_test_scaled)

In [ ]:
# Calculăm metricle
ridge_mse = mean_squared_error(y_test, ridge_pred)
ridge_rmse = np.sqrt(ridge_mse)
ridge_mae  = mean_absolute_error(y_test, ridge_pred)
ridge_r2   = r2_score(y_test, ridge_pred)

In [ ]:
#5.3 Lasso Regression cu cross‑validation
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-04, 1.20450354e-04, 1.45082878e-04, 1.74752840e-04,
       2.10490414e-04, 2.53536449e-04, 3.05385551e-04, 3.67837977e-04,
       4.43062146e-04, 5.33669923e-04, 6.42807312e-04, 7.74263683e-04,
       9.32603347e-04, 1.12332403e-03, 1.35304777e-03, 1.62975083e-03,
       1.96304065e-03, 2.36448941e-03, 2.84803587e-03, 3.43046929e-03,
       4.13201240e-03, 4.97702356e-0...
       1.38488637e+02, 1.66810054e+02, 2.00923300e+02, 2.42012826e+02,
       2.91505306e+02, 3.51119173e+02, 4.22924287e+02, 5.09413801e+02,
       6.13590727e+02, 7.39072203e+02, 8.90215085e+02, 1.07226722e+03,
       1.29154967e+03, 1.55567614e+03, 1.87381742e+03, 2.25701972e+03,
       2.71858824e+03, 3.27454916e+03, 3.94420606e+03, 4.75081016e+03,
       5.72236766e+03, 6.89261210e+03, 8.30217568e+03, 1.00000000e+04]),
        cv=5, random_state=42)

In [ ]:
lasso_alpha = lasso_cv.alpha_
print(f"Alpha optim pentru Lasso: {lasso_alpha:.5f}")

Alpha optim pentru Lasso: 0.24771


In [ ]:
lasso_pred = lasso_cv.predict(X_test_scaled)

In [ ]:
lasso_mse = mean_squared_error(y_test, lasso_pred) # Calculate MSE
lasso_rmse = np.sqrt(lasso_mse) # Calculate RMSE from MSE
lasso_mae  = mean_absolute_error(y_test, lasso_pred)
lasso_r2   = r2_score(y_test, lasso_pred)

In [ ]:
# Extragem coeficienții
lasso_coeffs = pd.Series(lasso_cv.coef_, index=feature_cols)
print("Coeficienții Lasso (pe date standardizate):")
print(lasso_coeffs)

Coeficienții Lasso (pe date standardizate):
X1     2.692748
X2    -3.612372
X3    -9.383899
X4    20.852458
X5    20.300745
D      3.147563
dtype: float64


In [ ]:
#5.4 Elastic Net Regression cu cross‑validation
# Definim gama de proporții L1 (l1_ratio) între Lasso (1) și Ridge (0)
l1_ratios = np.linspace(0.1, 1.0, 10)  # 10 valori între 0.1 și 1.0

In [ ]:
enet_cv = ElasticNetCV(l1_ratio=l1_ratios, alphas=alphas, cv=5, random_state=42)
enet_cv.fit(X_train_scaled, y_train)

ElasticNetCV(alphas=array([1.00000000e-04, 1.20450354e-04, 1.45082878e-04, 1.74752840e-04,
       2.10490414e-04, 2.53536449e-04, 3.05385551e-04, 3.67837977e-04,
       4.43062146e-04, 5.33669923e-04, 6.42807312e-04, 7.74263683e-04,
       9.32603347e-04, 1.12332403e-03, 1.35304777e-03, 1.62975083e-03,
       1.96304065e-03, 2.36448941e-03, 2.84803587e-03, 3.43046929e-03,
       4.13201240e-03, 4.977023...
       2.91505306e+02, 3.51119173e+02, 4.22924287e+02, 5.09413801e+02,
       6.13590727e+02, 7.39072203e+02, 8.90215085e+02, 1.07226722e+03,
       1.29154967e+03, 1.55567614e+03, 1.87381742e+03, 2.25701972e+03,
       2.71858824e+03, 3.27454916e+03, 3.94420606e+03, 4.75081016e+03,
       5.72236766e+03, 6.89261210e+03, 8.30217568e+03, 1.00000000e+04]),
             cv=5,
             l1_ratio=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
             random_state=42)

In [ ]:
enet_alpha    = enet_cv.alpha_
enet_l1_ratio = enet_cv.l1_ratio_
print(f"Alpha optim pentru ElasticNet: {enet_alpha:.5f}, l1_ratio optim: {enet_l1_ratio:.2f}")

Alpha optim pentru ElasticNet: 0.43288, l1_ratio optim: 0.10


In [ ]:
enet_pred = enet_cv.predict(X_test_scaled)

In [ ]:
enet_mse = mean_squared_error(y_test, enet_pred)
enet_rmse = np.sqrt(enet_mse)
enet_mae  = mean_absolute_error(y_test, enet_pred)
enet_r2   = r2_score(y_test, enet_pred)

In [ ]:
#6 Calculul unor indicatori suplimentari (MAPE) și compararea modelelor
# Definim funcția MAPE (Mean Absolute Percentage Error)
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# Compilăm rezultatele într-un DataFrame pentru comparație
results = pd.DataFrame({
    'Model': ['Ridge', 'Lasso', 'ElasticNet'],
    'Alpha' : [ridge_alpha, lasso_alpha, enet_alpha],
    'L1_ratio' : [np.nan, np.nan, enet_l1_ratio],
    'RMSE' : [ridge_rmse, lasso_rmse, enet_rmse],
    'MAE'  : [ridge_mae, lasso_mae, enet_mae],
    'MAPE' : [mape(y_test, ridge_pred), mape(y_test, lasso_pred), mape(y_test, enet_pred)],
    'R2'   : [ridge_r2, lasso_r2, enet_r2]
})

In [ ]:
# Afisăm tabelul cu rezultate
display(results)

,Model,Alpha,L1_ratio,RMSE,MAE,MAPE,R2
0,Ridge,31.257158,NaN,26.790890,21.949477,55.557386,0.050697
1,Lasso,0.247708,NaN,28.193128,22.667262,56.998375,-0.051277
2,ElasticNet,0.432876,0.1,26.591354,21.850806,55.326086,0.064785


In [ ]:
#7. Modele ML
#7.1 Random Forest Regressor cu GridSearch
# Definim modelul de bază
rf = RandomForestRegressor(random_state=42)

In [ ]:
# Definim o grilă de hiperparametri
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 3, 5, 7],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2]


In [ ]:
# Folosim GridSearchCV pentru a găsi cea mai bună combinație
rf_grid = GridSearchCV(rf, rf_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
rf_grid.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 3, 5, 7],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 4],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [ ]:
# Parametrii optimi
print("Parametrii optimi pentru Random Forest:", rf_grid.best_params_)


Parametrii optimi pentru Random Forest: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}


In [ ]:
# Predicții și indicatori
rf_pred = rf_grid.predict(X_test_scaled)
rf_mse = mean_squared_error(y_test, rf_pred)
rf_rmse = np.sqrt(rf_mse)
rf_mae  = mean_absolute_error(y_test, rf_pred)
rf_r2   = r2_score(y_test, rf_pred)

In [ ]:
# Importanța variabilelor
feature_importance = pd.Series(rf_grid.best_estimator_.feature_importances_, index=feature_cols)
print("Importanța predictorilor în Random Forest:")
print(feature_importance.sort_values(ascending=False))

Importanța predictorilor în Random Forest:
X4    0.512118
X5    0.200212
X1    0.121984
X3    0.093665
X2    0.069910
D     0.002111
dtype: float64


In [ ]:
#7.2 Gradient Boosting Regressor (GBR)
gbr = GradientBoostingRegressor(random_state=42)

gbr_params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [2, 3, 4]
}

In [ ]:
gbr_grid = GridSearchCV(gbr, gbr_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
gbr_grid.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [2, 3, 4], 'n_estimators': [100, 200]},
             scoring='neg_mean_squared_error')

In [ ]:
print("Parametrii optimi pentru Gradient Boosting:", gbr_grid.best_params_)

Parametrii optimi pentru Gradient Boosting: {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 100}


In [ ]:
gbr_pred = gbr_grid.predict(X_test_scaled)
gbr_rmse = mean_squared_error(y_test, gbr_pred)
gbr_mae  = mean_absolute_error(y_test, gbr_pred)
gbr_r2   = r2_score(y_test, gbr_pred)

In [ ]:
#7.3 Support Vector Regression (SVR)
svr = SVR(kernel='rbf')

svr_params = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.5],
    'gamma': ['scale', 'auto']
}

In [ ]:
svr_grid = GridSearchCV(svr, svr_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
svr_grid.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'epsilon': [0.01, 0.1, 0.5],
                         'gamma': ['scale', 'auto']},
             scoring='neg_mean_squared_error')

In [ ]:
print("Parametrii optimi pentru SVR:", svr_grid.best_params_)


Parametrii optimi pentru SVR: {'C': 100, 'epsilon': 0.5, 'gamma': 'auto'}


In [ ]:
svr_pred = svr_grid.predict(X_test_scaled)
svr_rmse = mean_squared_error(y_test, svr_pred)
svr_mae  = mean_absolute_error(y_test, svr_pred)
svr_r2   = r2_score(y_test, svr_pred)

In [ ]:
#8. Compilarea finală a rezultatelor
# Creăm un DataFrame final cu metrici pentru toate modelele
final_results = pd.DataFrame({
    'Model': ['Ridge', 'Lasso', 'ElasticNet', 'RandomForest', 'GradientBoosting', 'SVR'],
    'RMSE': [ridge_rmse, lasso_rmse, enet_rmse, rf_rmse, gbr_rmse, svr_rmse],
    'MAE' : [ridge_mae, lasso_mae, enet_mae, rf_mae, gbr_mae, svr_mae],
    'R2'  : [ridge_r2, lasso_r2, enet_r2, rf_r2, gbr_r2, svr_r2]
})
display(final_results)

,Model,RMSE,MAE,R2
0,Ridge,26.790890,21.949477,0.050697
1,Lasso,28.193128,22.667262,-0.051277
2,ElasticNet,26.591354,21.850806,0.064785
3,RandomForest,30.529657,26.695878,-0.232748
4,GradientBoosting,759.426186,24.222604,-0.004422
5,SVR,767.840239,22.054738,-0.015550
